In [6]:
import pandas as pd
import gzip
import json
import polars as pl

In [3]:
with open(
    "/cluster/home/tsomeya/projects/impossible_inherent_entropy/results/length_sampling/100M_samples_eos_zipf_min1_max20/samples.txt",
    "r",
    encoding="utf-8",
) as f:
    sentences = [next(f).strip() for _ in range(1_000)]

In [28]:
with gzip.open(
    "/cluster/home/tsomeya/projects/impossible_inherent_entropy/results/length_sampling/100_samples_eos_zipf_min1_max20/sample_counts.gz",
    "rt",
    encoding="utf-8",
) as f:
    sentence_counts = json.load(f)

In [29]:
sentence_counts

{'appalate sub pi ob bullegate rel amackists ob rebinked [eos]': 1,
 'businators sub fusilitateda [eos]': 1,
 'la sub mattakateda sa sedundifieda rel appalates sub forigateda [eos]': 1,
 'ampercial ob gratened rel ambather ob phonorizeda rel ballisons sub prask amackist ob bunessisteda [eos]': 1,
 'mangerizeda rel amackists sub appalate ob gratenizeda sa fanched rel povician sub tawlerized [eos]': 1,
 'lustodist sub prask amationist ob bactivitated rel ambather sub remiffs sa squail rel amationists ob yolved [eos]': 1,
 'si sub stremord ampercial sub spange blopes ob tirastified sa lurchified rel wolarist sub peaserizes sa pactess [eos]': 1,
 'ja sub fusilitates rel autoner ob yolveda rel amationists ob intrinses [eos]': 1,
 'pi sub crene amationists ob greanerized [eos]': 1,
 'bo ca amackists ob intrinseda da damerticateda rel ballisons ob garfs rel amackist ob gratenizes rel appalate sub saffles [eos]': 1,
 'denaticateda rel appalates sub remiffeda [eos]': 1,
 'duppist sub me sub spe

In [13]:
df = pl.DataFrame(
    {"sentence": list(sentence_counts.keys()), "count": list(sentence_counts.values())}
)

In [44]:
df = pl.read_csv(
    "/cluster/home/tsomeya/projects/impossible_inherent_entropy/results/length_sampling/100_samples_eos_zipf_min1_max20/true_prob.json.gz",
    new_columns=["sentence", "count"],
)

In [45]:
df

sentence,count,true_log_prob,true_prob
str,i64,f64,f64
"""businators sub fusilitateda [e…",1,1.0,2.718282
"""appalate sub pi ob bullegate r…",1,1.0,2.718282
"""         """,null,null,null


In [4]:
# df = (
#     pl.scan_csv(
#         "/cluster/home/tsomeya/projects/impossible_inherent_entropy/results/length_sampling/base-grammar_eos_zipf/samples.txt.gz",
#         separator="\n",
#         has_header=False,
#         new_columns=["sentence"],
#     )
#     .collect()
#     .with_columns(
#         pl.col("sentence").str.strip_chars()
#     )  # Use str.strip_chars() instead of str.strip()
#     .group_by("sentence")
#     .agg(pl.count().alias("count"))
# )

/tmp/ipykernel_2907062/59962965.py:13: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("count"))


In [15]:
count_distribution = (
    df.group_by("count").agg(pl.count().alias("frequency")).sort("count")
)
print(count_distribution)

shape: (1, 2)
┌───────┬───────────┐
│ count ┆ frequency │
│ ---   ┆ ---       │
│ i64   ┆ u32       │
╞═══════╪═══════════╡
│ 1     ┆ 100       │
└───────┴───────────┘


/tmp/ipykernel_2907062/1311824965.py:2: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  df.group_by("count").agg(pl.count().alias("frequency")).sort("count")


In [16]:
target_samples_per_count = 100

# 各countグループからサンプリング
balanced_df = (
    df.group_by("count")
    .agg(
        pl.col("sentence").sample(
            n=pl.when(pl.count() >= target_samples_per_count)
            .then(target_samples_per_count)
            .otherwise(pl.count())
        )
    )
    .sort("count")
)

/tmp/ipykernel_2907062/608659407.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  n=pl.when(pl.count() >= target_samples_per_count)
/tmp/ipykernel_2907062/608659407.py:10: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .otherwise(pl.count())


In [17]:
balanced_df

count,sentence
i64,list[str]
1,"[""appalate sub pi ob bullegate rel amackists ob rebinked [eos]"", ""businators sub fusilitateda [eos]"", … ""bo sub flingericians ob skeezes rel jonter ob baniticates sa engarified rel ballison ob gratenize rel lustodists sub fusilitate [eos]""]"


In [18]:
sampled_dist = (
    balanced_df.explode("sentence")
    .group_by("count")
    .agg(pl.count("sentence").alias("sampled"))
    .sort("count")
)
print("\nSamples obtained from each count group:")
print(sampled_dist)


Samples obtained from each count group:
shape: (1, 2)
┌───────┬─────────┐
│ count ┆ sampled │
│ ---   ┆ ---     │
│ i64   ┆ u32     │
╞═══════╪═════════╡
│ 1     ┆ 100     │
└───────┴─────────┘


In [19]:
sampled_sentences = balanced_df.get_column("sentence").explode().to_list()

In [20]:
len(sampled_sentences)

100

In [26]:
with gzip.open(
    "/cluster/home/tsomeya/projects/impossible_inherent_entropy/results/length_sampling/100M_samples_eos_zipf_min1_max20/filtered_samples.txt.gz",
    "rt",
    encoding="utf-8",
) as f:
    sentences = f.readlines()

In [27]:
len(sentences)

26442